----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - EndPoint
This notebook demonstrates how to retrieve data from the Refinitiv Data Platform using EndPoint objects of the low level Delivery layer of the Refinitiv Data library.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.delivery import endpoint_request
import asyncio

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session("platform.rdp")

<refinitiv.data.session.platform.Definition object at 0x7fb54f310c10 {session_name='rdp'}>

## Retrieve data

### Defining an Endpoint
At the heart of the Endpoint Request interfaces, is the data endpoint URL.  For example, the following data services are represented by their endpoint URLs:

* Historical Pricing: https://api.refinitiv.com/data/historical-pricing/v1/views/events
* News Headlines: https://api.refinitiv.com/data/news/v1/headlines 
* Symbology Lookup: https://api.refinitiv.com/discovery/symbology/v1/lookup 
* Basic ESG scores: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic

When creating a request to retrieve data from an endpoint, the first step is to specify the endpoint URL, for example:

In [4]:
esg_url = 'https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic'

#### Defining Endpoint Request Properties
Depending on the service and how you choose to prepare your endpoint URL, the interfaces support the ability to specify additional properties to be included within your request.  The following request features are available:

* Query Parameters Endpoint: URLs supporting query strings are represented as field=value pairs.
Eg: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic?universe=IBM.N

* Path Parameters : In some cases, URLs define parameters that are embedded within the URL.  When defining the URL as part of your application, you must enclose the path parameter(s) within curly braces.
Eg: https://api.refinitiv.com/data/historical-pricing/v1/views/events/{universe}

* Header Parameters : Endpoints that enforce different content types or formats, typically require the specification within the header portion of the HTTP request.  For example, the endpoint to retrieve a news story offers the ability to specify the response format as either "text/html" or "application/json", which is specified within the HTTP header of the request.

* Body Parameters : Many of the RDP endpoints support the HTTP POST defining the ability to specify a body of data in the request.  The libraries support the ability to specify JSON details within the support endpoint request.  

The following simple code segments demonstrate the above mechanisms.

**Query Parameter:**

In [5]:
# Example request with Query Parameter - ESG

request_definition = rd.delivery.endpoint_request.Definition(
    url = esg_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "IBM.N"}
)
response = request_definition.get_data()
response.data.raw

{'links': {'count': 1},
 'variability': 'fixed',
 'universe': [{'Instrument': 'IBM.N',
   'Company Common Name': 'International Business Machines Corp',
   'Organization PermID': '4295904307',
   'Reporting Currency': 'USD'}],
 'data': [['IBM.N',
   '2020-12-31',
   96.625022435838,
   51.5117088466531,
   78.9800773596806,
   98.9825581395348,
   99.3093922651933,
   53.8135593220339,
   99.6099843993759,
   83.3850931677018,
   99.4539781591263,
   98.1574539363484,
   39.6890096618357,
   78.06461352657,
   70.795847750865,
   73.3005404816844,
   100,
   '2021-08-28T00:00:00']],
 'messages': {'codes': [[-1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1]],
  'descriptions': [{'code': -1, 'description': 'ok'}]},
 'headers': [{'name': 'instrument',
   'title': 'Instrument',
   'type': 'string',
   'description': 'The requested Instrument as defined by the user.'},
  {'name': 'periodenddate',
   't

**Path Parameter:**

In [6]:
# Example request with Path parameter - News Story
request_definition = rd.delivery.endpoint_request.Definition(
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}',               # Specify our Endpoint URL with a path parameter placeholder named 'storyId' 
    path_parameters = {"storyId": "urn:newsml:reuters.com:20210805:nZON003MH6:1"}   # Specify the 'storyId' path parameter value
)
response = request_definition.get_data()
response.data.raw

{'newsItem': {'_conformance': 'power',
  '_guid': 'tag:reuters.com,2021:newsml_ZON003MH6',
  '_standard': 'NewsML-G2',
  '_standardversion': '2.18',
  '_version': 2,
  '_xml:lang': 'fr',
  'catalogRef': [{'_href': 'http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml'}],
  'rightsInfo': [{'_xml:lang': 'en',
    'copyrightHolder': {'_literal': 'Thomson Reuters'},
    'copyrightNotice': [{'$': '(c) Copyright Thomson Reuters 2021. Click For Restrictions - https://agency.reuters.com/en/copyright.html'}]}],
  'itemMeta': {'itemClass': {'_qcode': 'ninat:text', '_rtr:msgType': 'S'},
   'provider': {'_literal': 'reuters.com'},
   'versionCreated': {'$': '2021-08-05T12:48:13Z'},
   'firstCreated': {'$': '2021-08-05T12:48:13Z'},
   'pubStatus': {'_qcode': 'stat:usable'},
   'role': {'_qcode': 'itemRole:B'},
   'fileName': {'$': '2021-08-05T124813Z_2_ZON003MH6_RTRLYNXT_0_USA-CHOMAGE-INSCRIPTIONS.XML'},
   'profile': {'_versioninfo': '00.00.01', '$': 'SNI-Tex

**Header Parameter:**

In [7]:
# Example request with Path + Header parameter - News Story
request_definition = rd.delivery.endpoint_request.Definition(
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}',               # Specify our Endpoint URL with a path parameter placeholder named 'storyId' 
    path_parameters = {"storyId": "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"},  # Specify the 'storyId' path parameter value
    header_parameters = {"accept": "text/html"}                                     # Specify an additional header parameter
)
response = request_definition.get_data()
response.data.raw

{'newsItem': {'_conformance': 'power',
  '_guid': 'tag:reuters.com,2021-03-30:newsml_HKS9WjjsZ',
  '_standard': 'NewsML-G2',
  '_standardversion': '2.18',
  '_version': 2,
  'catalogRef': [{'_href': 'http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml'}],
  'rightsInfo': [{'copyrightHolder': {'_literal': ''},
    'copyrightNotice': [{}]}],
  'itemMeta': {'itemClass': {'_qcode': 'ninat:text', '_rtr:msgType': 'R'},
   'provider': {'_literal': 'reuters.com'},
   'versionCreated': {'$': '2021-03-30T14:09:48.681Z'},
   'firstCreated': {'$': '2021-03-30T14:09:09.503Z'},
   'pubStatus': {'_qcode': 'stat:usable'},
   'role': {'_qcode': 'itemRole:N'},
   'title': [{'$': 'Shenwan Hongyuan Group Co Ltd - An announcement has just been published by the issuer in the Chinese section of this website, a corresponding version of which may or may not be published in this section(with URL)'}],
   'signal': [{'_qcode': 'edStat:U'}],
   'expires': [{'$': '2022-04-30T

**Body Parameter:**

In [8]:
# Example request with Body Parameter - Symbology Lookup
request_definition = rd.delivery.endpoint_request.Definition(
    method = rd.delivery.endpoint_request.RequestMethod.POST,
    url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup',
    body_parameters = {         # Specify body parameters
        "from": [
            {
                "identifierTypes": [
                    "RIC"
                ],
                "values": [
                    "MSFT.O",
                    "IBM.N"
                ]
            }
        ],
        "to": [
            {
                "identifierTypes": [
                    "LEI",
                    "ISIN",
                    "ExchangeTicker"
                ]
            }
        ],
        "type": "auto"
    }
)
response = request_definition.get_data()
response.data.raw

{'data': [{'input': [{'value': 'IBM.N', 'identifierType': 'RIC'}],
   'output': [{'value': 'US4592001014', 'identifierType': 'ISIN'},
    {'value': 'IBM', 'identifierType': 'ExchangeTicker'},
    {'value': 'VGRQXHF3J8VDLUA7XE92', 'identifierType': 'LEI'}]},
  {'input': [{'value': 'MSFT.O', 'identifierType': 'RIC'}],
   'output': [{'value': 'INR2EJN1ERAN0W5ZP974', 'identifierType': 'LEI'},
    {'value': 'US5949181045', 'identifierType': 'ISIN'},
    {'value': 'MSFT', 'identifierType': 'ExchangeTicker'}]}],
 'requestId': '8e52396e-55ec-42ae-a21a-da0993298d8b',
 'effectiveAt': '2021-12-16T16:01:18.675Z',
 'messages': []}

## Close the session

In [9]:
rd.close_session()